# import modules

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import date
pd.options.display.max_columns = None

# import data

In [2]:
# import excel file
try:
    workbookName='input_template.xlsx'
    sheetName='input'
    import_data_raw=pd.read_excel(workbookName,
                          dtype=np.str,
                         sheet_name=sheetName)
    colsExcpected=['supply_of','fictitious_supply_by_marketplace','origin_country','origin_country_vat_id',
                 'origin_country_tax_number','destination_country','vat_rate','net_amount']
    actualCols=list(import_data_raw.columns)
    if colsExcpected==actualCols:
        pass
    else:
        print('worksheet {} of excel file {} could not be loaded (wrong columns)'.format(sheetName,workbookName))
        del import_data_raw        
except:
    print('worksheet {} of excel file {} could not be loaded (probably wrong names or file not in same folder as workbook)'.format(sheetName,workbookName))
    
# all EU Member States (Greece as 'GR' and 'EL')
eu_ms=['AT','BE','BG','HR','CY','CZ','DK','EE','FI','FR','DE','GR','EL',
       'HU','IE','IT','LV','LT','LU','MT','MT','NL','PL','PT','RO','SK',
       'SI','ES','SE']

# all standard vat rates in the EU
vat_rates_eu={'AT':0.2,'BE':0.21,'BG':0.2,'HR':0.25,'CY':0.19,'CZ':0.21,'DK':0.25,
              'EE':0.2,'FI':0.24,'FR':0.2,'DE':0.19,'GR':0.24,'EL':0.24,'HU':0.27,'IE':0.23,
              'IT':0.22,'LV':0.21,'LT':0.21,'LU':0.17,'MT':0.18,'NL':0.21,'PT':0.23,
              'PL':0.23,'RO':0.19,'SK':0.2,'SI':0.22,'ES':0.21,'SE':0.25}

df_vat_rates_eu=pd.DataFrame.from_dict(vat_rates_eu,orient='index')
df_vat_rates_eu=df_vat_rates_eu.reset_index()
df_vat_rates_eu=df_vat_rates_eu.rename(columns={0:'vat_rate','index':'country'})
df_vat_rates_eu['vat_rate_type']='STANDARD'
df_vat_rates_eu['vat_rate']=df_vat_rates_eu['vat_rate']*100

In [3]:
#''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
# Data verification: check import_data for errors and print errors to excel  
#''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
# date to be used

import_data=import_data_raw.copy()
dateTobeUsed=date.today()
dateTobeUsed=dateTobeUsed.strftime("%Y%m%d")

def change_comma_dot(value):
  try:
    try:
        return float(value)
    except:
        value=value.replace(',','.')
        return float(value)
  except:
    return np.nan
import_data['vat_rate']=import_data['vat_rate'].apply(lambda x: change_comma_dot(x))
import_data['net_amount']=import_data['net_amount'].apply(lambda x: change_comma_dot(x))

import_data=pd.merge(import_data,
                  df_vat_rates_eu,
                  how='left',
                  left_on=['vat_rate','destination_country'],
                  right_on=['vat_rate','country'])
import_data['vat_rate_type']=import_data['vat_rate_type'].fillna(value='REDUCED')
import_data=import_data.drop(columns=['country'])

conditionlist=[~import_data['supply_of'].isin(['goods','services'])]
choicelist=['wrong input in column \'supply_of\': \''+import_data['supply_of']+'\'']
import_data['supply_of_error']=np.select(conditionlist,choicelist,default='')

conditionlist=[~import_data['fictitious_supply_by_marketplace'].isin(['1','0']),
               (import_data['fictitious_supply_by_marketplace']=='1')&(import_data['supply_of']=='services')]
choicelist=['wrong input in column \'fictitious_supply_by_marketplace\': \''+import_data['fictitious_supply_by_marketplace']+'\'',
            'wrong input in column \'fictitious_supply_by_marketplace\': \''+import_data['fictitious_supply_by_marketplace']+'\'']
import_data['marketplace_error']=np.select(conditionlist,choicelist,default='')

conditionlist=[~import_data['origin_country'].isin(eu_ms)]
choicelist=['no EU Member State in column \'origin_country\': \''+import_data['origin_country']+'\'']
import_data['origin_country_error']=np.select(conditionlist,choicelist,default='')

conditionlist=[(import_data['origin_country_vat_id'].str[:2]!=import_data['origin_country'])&((~import_data['origin_country_vat_id'].isna())&(import_data['origin_country_vat_id']!=""))]
choicelist=['review column \'origin_country_vat_id\' (no match with origin_country): \''+import_data['origin_country_vat_id']+'\'']
import_data['origin_country_vat_id_error']=np.select(conditionlist,choicelist,default='')

conditionlist=[~import_data['origin_country_vat_id'].isna()&~import_data['origin_country_tax_number'].isna(),
              import_data['origin_country_vat_id'].isna()&import_data['origin_country_tax_number'].isna()]
choicelist=['please provide only one input: vat id or tax number. If vat id available, only the vat id is required',
           'please provide one input: vat id or tax number']
import_data['vat_id_error_tax_number']=np.select(conditionlist,choicelist,default='')

conditionlist=[~import_data['destination_country'].isin(eu_ms)]
choicelist=['no EU Member State in column \'destination_country\': \''+import_data['destination_country']+'\'']
import_data['destination_country_error']=np.select(conditionlist,choicelist,default='')

conditionlist=[import_data['destination_country']==import_data['origin_country']]
choicelist=['wrong input in column \'destination_country\' or \'origin_country\'. Countries are the same']
import_data['origin_destination_country_error']=np.select(conditionlist,choicelist,default='')

conditionlist=[import_data['vat_rate'].isna()]
choicelist=['no or wrong input in column \'vat_rate\'']
import_data['vat_rate_error']=np.select(conditionlist,choicelist,default='')

conditionlist=[import_data['net_amount'].isna()]
choicelist=['no or wrong input in column \'net_amount\'']
import_data['net_amount_error']=np.select(conditionlist,choicelist,default='')

import_data['all_errors']=import_data['supply_of_error']+import_data['origin_country_error']+import_data['marketplace_error']+import_data['origin_destination_country_error']+import_data['origin_country_vat_id_error']+import_data['vat_id_error_tax_number']+import_data['destination_country_error']+import_data['vat_rate_error']+import_data['net_amount_error']
conditionlist=[import_data['all_errors']=='']
choicelist=['']
import_data['errors']=np.select(conditionlist,choicelist,default='errors')
import_data=import_data.drop(columns=['all_errors'])
error_data=import_data[import_data['errors']=='errors'].copy()
error_data=error_data.reset_index()
error_data=error_data.rename(columns={'index':'import_row'})
error_data['import_row']=error_data['import_row']+2
error_data.to_excel('import_errors_'+dateTobeUsed+'.xlsx',index=False)


#''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
# continue w/o faulty data and create the individual "Satzarts"
#''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''

## adjust data
import_data_adj=import_data[import_data['errors']!='errors'][['supply_of','fictitious_supply_by_marketplace','origin_country','origin_country_vat_id',
                                                              'origin_country_tax_number','destination_country','vat_rate','net_amount','vat_rate_type']].copy()

## alternative: include the data identified as faulty as well
# import_data_adj=import_data[['supply_of','fictitious_supply_by_marketplace','origin_country','origin_country_vat_id',
#                             'origin_country_tax_number','destination_country','vat_rate','net_amount','vat_rate_type']].copy()

import_data_adj['vat_amount']=import_data_adj['net_amount']*import_data_adj['vat_rate']/100

## adjust fictitious_supply_by_marketplace
conditionlist=[import_data_adj['fictitious_supply_by_marketplace']=='0']
choicelist=['1']
import_data_adj['fictitious_supply_by_marketplace']=np.select(conditionlist,choicelist,default='2')


## Land des Verbrauchs (Satzart 1)
ship_to_unique=list(import_data_adj['destination_country'].unique())
satz1_final=pd.DataFrame(ship_to_unique,columns=['import_data_adj'])
satz1_final['Satzart']='1'
satz1_final=satz1_final[['Satzart','import_data_adj']]
print('\nLand des Verbrauchs (Satzart 1), first 3 rows')
display(satz1_final.head(3))

## Umsätze des Steuerpflichtigen - Dienstleistungen (Satzart 2)
satz2_all=import_data_adj[(import_data_adj['origin_country']=='DE')&
                          (import_data_adj['supply_of']=='services')][['destination_country','vat_rate','net_amount','vat_amount','vat_rate_type']]
satz2_all['Satzart']='2'

satz2_all=satz2_all[['Satzart','destination_country','vat_rate_type','vat_rate','net_amount','vat_amount']]
satz2_final=(satz2_all.groupby(by=['Satzart','destination_country','vat_rate_type','vat_rate'],
                        dropna=False,
                        as_index=False)
                        .agg({'net_amount':'sum','vat_amount': 'sum'})
                        .round(2)).copy()
print('\n\nUmsätze des Steuerpflichtigen - Dienstleistungen (Satzart 2), first 3 rows')
display(satz2_final.head(3))

## Umsätze für vom Inland aus durchgeführte Warenlieferungen (Satzart 3)
satz3_all=import_data_adj[(import_data_adj['origin_country']=='DE')&
                          (import_data_adj['supply_of']=='goods')][['destination_country','vat_rate','net_amount','vat_amount','vat_rate_type']]
satz3_all['Satzart']='3'

satz3_all=satz3_all[['Satzart','destination_country','vat_rate_type','vat_rate','net_amount','vat_amount']]
satz3_final=(satz3_all.groupby(by=['Satzart','destination_country','vat_rate_type','vat_rate'],
                        dropna=False,
                        as_index=False)
                        .agg({'net_amount':'sum','vat_amount': 'sum'})
                        .round(2)).copy()
print('\n\nUmsätze für vom Inland aus durchgeführte Warenlieferungen (Satzart 3), first 3 rows')
display(satz3_final)

## Umsätze einer festen Niederlassung - Dienstleistungen (Satzart 4)
satz4_all=import_data_adj[(import_data_adj['origin_country']!='DE')&
                          (import_data_adj['supply_of']=='services')][['destination_country','supply_of','origin_country','origin_country_vat_id','origin_country_tax_number','vat_rate','net_amount','vat_amount','vat_rate_type']]
satz4_all['Satzart']='4'

satz4_all=satz4_all[['Satzart','destination_country','origin_country',
                     'origin_country_vat_id','origin_country_tax_number','vat_rate_type','vat_rate','net_amount','vat_amount']]
satz4_final=(satz4_all.groupby(by=['Satzart','destination_country','origin_country',
                                 'origin_country_vat_id','origin_country_tax_number','vat_rate_type','vat_rate'],
                        dropna=False,
                        as_index=False)
                        .agg({'net_amount':'sum','vat_amount': 'sum'})
                        .round(2)).copy()
print('\n\nUUmsätze einer festen Niederlassung - Dienstleistungen (Satzart 4), first 3 rows')
display(satz4_final.head(3))

## Umsätze für aus anderen Mitgliedstaaten durchgeführte Warenlieferungen (Satzart 5)
satz5_all=import_data_adj[(import_data_adj['origin_country']!='DE')&
                          (import_data_adj['supply_of']=='goods')][['destination_country','fictitious_supply_by_marketplace','supply_of','origin_country','origin_country_vat_id','origin_country_tax_number','vat_rate','net_amount','vat_amount','vat_rate_type']]
satz5_all['Satzart']='5'

satz5_all=satz5_all[['Satzart','destination_country','fictitious_supply_by_marketplace','origin_country',
                     'origin_country_vat_id','origin_country_tax_number','vat_rate_type','vat_rate','net_amount','vat_amount']]
satz5_final=(satz5_all.groupby(by=['Satzart','destination_country','fictitious_supply_by_marketplace','origin_country',
                                 'origin_country_vat_id','origin_country_tax_number','vat_rate_type','vat_rate'],
                        dropna=False,
                        as_index=False)
                        .agg({'net_amount':'sum','vat_amount': 'sum'})
                        .round(2)).copy()
print('\n\nUmsätze für aus anderen Mitgliedstaaten durchgeführte Warenlieferungen (Satzart 5), first 3 rows')
display(satz5_final.head(3))

#''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
## concat all Satzarts to create final csv
#''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''

#concat
df_final_header=pd.concat([satz1_final,
                           satz2_final,
                           satz3_final,
                           satz4_final,
                           satz5_final], ignore_index=True)

version=pd.DataFrame(data={0: ['#v1.0']})
satz1_final_adj=satz1_final.copy()
satz1_final_adj.columns = np.arange(len(satz1_final_adj.columns))
satz2_final_adj=satz2_final.copy()
satz2_final_adj.columns = np.arange(len(satz2_final_adj.columns))
satz3_final_adj=satz3_final.copy()
satz3_final_adj.columns = np.arange(len(satz3_final_adj.columns))
satz4_final_adj=satz4_final.copy()
satz4_final_adj.columns = np.arange(len(satz4_final_adj.columns))
satz5_final_adj=satz5_final.copy()
satz5_final_adj.columns = np.arange(len(satz5_final_adj.columns))

df_final=pd.concat([version,
                    satz1_final_adj,
                    satz2_final_adj,
                    satz3_final_adj,
                    satz4_final_adj,
                    satz5_final_adj], ignore_index=True)

## create csv

#make final copy
df_final_to_be_printed=df_final.copy()
df_final_to_be_printed=df_final_to_be_printed.fillna('')
#change dot to comma and save as text with two digits (even though it is mentioned in the BZSt csv upload guidline, that floats must come
#with dot as separator, it produced errors at least when I tried it)
def change_format(x):
  if x is not np.nan:
    try:
      x=str(x)
      splitList=x.split('.')
      comma=splitList[1].ljust(2,'0')
      x=splitList[0]+','+comma
      return x
    except:
      return x
  else:
    return x

df_final_to_be_printed[3]=df_final_to_be_printed[3].apply(lambda x:change_format(x))
df_final_to_be_printed[4]=df_final_to_be_printed[4].apply(lambda x:change_format(x))
df_final_to_be_printed[5]=df_final_to_be_printed[5].apply(lambda x:change_format(x))
df_final_to_be_printed[6]=df_final_to_be_printed[6].apply(lambda x:change_format(x))
df_final_to_be_printed[7]=df_final_to_be_printed[7].apply(lambda x:change_format(x))
df_final_to_be_printed[8]=df_final_to_be_printed[8].apply(lambda x:change_format(x))
df_final_to_be_printed[9]=df_final_to_be_printed[9].apply(lambda x:change_format(x))
df_final_to_be_printed.to_csv('oss_upload_'+dateTobeUsed+'.csv',
                header=False,
                index=False)

print('\n\nfinal summary per destination_country')
(df_final_header.groupby(by=['destination_country'],
                        dropna=True,
                        as_index=False)
                        .agg({'net_amount':'sum','vat_amount': 'sum'})
                        .round(2)).copy()


Land des Verbrauchs (Satzart 1), first 3 rows


,Satzart,import_data_adj
0,1,DK
1,1,DE
2,1,PT




Umsätze des Steuerpflichtigen - Dienstleistungen (Satzart 2), first 3 rows


,Satzart,destination_country,vat_rate_type,vat_rate,net_amount,vat_amount
0,2,IE,STANDARD,23.0,500.0,115.0




Umsätze für vom Inland aus durchgeführte Warenlieferungen (Satzart 3), first 3 rows


...

,Satzart,destination_country,origin_country,origin_country_vat_id,origin_country_tax_number,vat_rate_type,vat_rate,net_amount,vat_amount
0,4,FR,CZ,CZXXXXXXXXXXXX,NaN,STANDARD,20.0,500.0,100.0
1,4,PT,PL,NaN,55555555555555504,STANDARD,23.0,500.0,115.0




Umsätze für aus anderen Mitgliedstaaten durchgeführte Warenlieferungen (Satzart 5), first 3 rows


,Satzart,destination_country,fictitious_supply_by_marketplace,origin_country,origin_country_vat_id,origin_country_tax_number,vat_rate_type,vat_rate,net_amount,vat_amount
0,5,DE,1,ES,ESNXXXXXXXXXXXX,NaN,STANDARD,19.0,200.0,38.0
1,5,FR,1,CZ,CZXXXXXXXXXXXX,NaN,STANDARD,20.0,500.0,100.0




final summary per destination_country


,destination_country,net_amount,vat_amount
0,DE,200.0,38.00
1,DK,119.9,29.98
2,FR,1000.0,200.00
3,IE,500.0,115.00
4,PT,500.0,115.00
